In [1]:
import pandas as pd, numpy as np
from pyfasttext import FastText
from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss

In [188]:
train = pd.read_csv('input/train.csv')
test = pd.read_csv('input/test.csv')

COMMENT = 'comment_text'
train[COMMENT].fillna("unknown", inplace=True)
test[COMMENT].fillna("unknown", inplace=True)
train[COMMENT] = train[COMMENT].str.lower().str.replace('https?:\/\/[^\s]*','').str.replace("’","'").str.replace("''",' ').str.replace("'"," ' ").str.replace('“','"').str.replace('”','"').str.replace('"',' : ').str.replace('.',' . ').str.replace(',',' , ').str.replace('[',' [ ').str.replace(']',' ] ').str.replace('(',' ( ').str.replace(')',' ) ').str.replace('!',' ! ').str.replace('?',' ? ').str.replace(';',' ').str.replace(':',' ').str.replace('-',' - ').str.replace('=', ' ').str.replace('=', ' ').str.replace('*', ' ').str.replace('|', ' ').str.replace('«', ' ').str.replace('\d', ' ').str.replace('\n', ' ').str.replace('\s\s+',' ').str.strip()
test[COMMENT] = test[COMMENT].str.lower().str.replace('https?:\/\/[^\s]*','').str.replace("’","'").str.replace("''",' ').str.replace("'"," ' ").str.replace('“','"').str.replace('”','"').str.replace('"',' : ').str.replace('.',' . ').str.replace(',',' , ').str.replace('[',' [ ').str.replace(']',' ] ').str.replace('(',' ( ').str.replace(')',' ) ').str.replace('!',' ! ').str.replace('?',' ? ').str.replace(';',' ').str.replace(':',' ').str.replace('-',' - ').str.replace('=', ' ').str.replace('=', ' ').str.replace('*', ' ').str.replace('|', ' ').str.replace('«', ' ').str.replace('\d', ' ').str.replace('\n', ' ').str.replace('\s\s+',' ').str.strip()

# remove all punctuation, a little bit worser
#train[COMMENT] = train[COMMENT].str.lower().str.replace('https?:\/\/[^\s]*','').str.replace("’","'").str.replace("''",'').str.replace("'","").str.replace('“','').str.replace('”','').str.replace('"','').str.replace('.','').str.replace(',','').str.replace('[','').str.replace(']','').str.replace('(','').str.replace(')','').str.replace('!','').str.replace('?','').str.replace(';','').str.replace(':','').str.replace('-','').str.replace('=', '').str.replace('=', '').str.replace('*', '').str.replace('|', '').str.replace('«', '').str.replace('\d', ' ').str.replace('\n', '').str.replace('\s\s+',' ').str.strip()
#test[COMMENT] = test[COMMENT].str.lower().str.replace('https?:\/\/[^\s]*','').str.replace("’","'").str.replace("''",'').str.replace("'","").str.replace('“','').str.replace('”','').str.replace('"','').str.replace('.','').str.replace(',','').str.replace('[','').str.replace(']','').str.replace('(','').str.replace(')','').str.replace('!','').str.replace('?','').str.replace(';','').str.replace(':','').str.replace('-','').str.replace('=', '').str.replace('=', '').str.replace('*', '').str.replace('|', '').str.replace('«', '').str.replace('\d', ' ').str.replace('\n', '').str.replace('\s\s+',' ').str.strip()

# test remove you/your (not work)
#train[COMMENT] = train[COMMENT].str.replace("your","").str.replace("you","")
#test[COMMENT] = test[COMMENT].str.replace("your","").str.replace("you","")

In [163]:
#train_test = train.copy()
#train_test[train_test[COMMENT].str.contains('27')][COMMENT].str.replace("’","'").str.replace("''",' ').str.replace("'"," ' ").str.replace('“','"').str.replace('”','"').str.replace('"',' : ').str.replace('.',' . ').str.replace(',',' , ').str.replace('(',' ( ').str.replace(')',' ) ').str.replace('!',' ! ').str.replace('?',' ? ').str.replace(';',' ').str.replace(':',' ').str.replace('-',' - ').str.replace('=', ' ').str.replace('=', ' ').str.replace('*', ' ').str.replace('|', ' ').str.replace('«', ' ').str.replace('\d', ' ').str.replace('\n', ' ').str.replace('\s\s+','').str.strip()

56999    ! ! ! !
Name: comment_text, dtype: object

In [189]:
label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
train['clean'] = 1-train[label_cols].max(axis=1)
train.describe()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate,clean
count,9.585100e+04,95851.000000,95851.000000,95851.000000,95851.000000,95851.000000,95851.000000,95851.000000
mean,4.994359e+11,0.096368,0.010068,0.053301,0.003182,0.049713,0.008492,0.897862
std,2.890136e+11,0.295097,0.099832,0.224635,0.056320,0.217352,0.091762,0.302831
min,2.225664e+07,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.473437e+11,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,5.001297e+11,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,7.501088e+11,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,9.999882e+11,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [190]:
train_df = None
val_df = None
for label in label_cols:
    train['label'] = ""
    train.loc[(train[label]==1), 'label'] = "__label__" + label + " "
    train.loc[(train[label]==0), 'label'] = "__label__clean" + " "
    train_df, val_df = train_test_split(train, test_size=0.2, random_state=1)
    train_df.to_csv("fasttext_train_"+label+".csv", columns=['label', 'comment_text'], index=False)    
    train.to_csv("fasttext_train_all_"+label+".csv", columns=['label', 'comment_text'], index=False)    

In [200]:
''' spliting comment doesn't help
def split_comment_text(comment_text):
    comments = comment_text.split(' . ')
    if len(comments) > 1:
        for i in range(len(comments)-1):
            if comments[i] != '.':
                comments[i] += ' .'
    sentences = []
    #sentences.append(comment_text)
    for comment in comments:
        if comment != '.':
            sentences.append(comment)
    return sentences

val_df['sentences'] = val_df['comment_text'].apply(lambda x: split_comment_text(x))
'''

"\ndef split_comment_text(comment_text):\n    comments = comment_text.split(' . ')\n    if len(comments) > 1:\n        for i in range(len(comments)-1):\n            if comments[i] != '.':\n                comments[i] += ' .'\n    sentences = []\n    #sentences.append(comment_text)\n    for comment in comments:\n        if comment != '.':\n            sentences.append(comment)\n    return sentences\n\nval_df['sentences'] = val_df['comment_text'].apply(lambda x: split_comment_text(x))\n"

In [193]:
def train_model(param, target, all_sample = False):
    model = FastText()
    #model.supervised(input='fasttext_train_'+target+'.csv', output=model_name(target, param), epoch=param["epoch"], minn=param["minn"], maxn=param["maxn"])
    if all_sample == False:
        model.supervised(input='fasttext_train_'+target+'.csv', output='parameter_search', epoch=param["epoch"], minn=param["minn"], maxn=param["maxn"], wordNgrams=param["wordNgrams"], dim=param["dim"], pretrainedVectors='comments_skipgram.vec')
    else:
        model.supervised(input='fasttext_train_all_'+target+'.csv', output='parameter_search', epoch=param["epoch"], minn=param["minn"], maxn=param["maxn"], wordNgrams=param["wordNgrams"], dim=param["dim"], pretrainedVectors='comments_skipgram.vec')
    return model

In [194]:
def calculate_log_loss(model, target):
    predict_probas = model.predict_proba(val_df[COMMENT],k=2) 
    pred = []
    for predict in predict_probas:
        pred_prob = 0
        for label, prob in predict:
            if label == target:         
                pred_prob = prob
        pred.append(pred_prob) 
    return log_loss(val_df[target],pred)

In [201]:
''' spliting comment doesn't help
import numpy as np
def calculate_log_loss_1(model, target):
    pred = []
    for index, row in val_df.iterrows():        
        predict_probas = model.predict_proba(row['sentences'],k=2) 
        #print(row['sentences'])
        #print(predict_probas)
        row_pred = []
        for predict in predict_probas:
            pred_prob = 0
            for label, prob in predict:
                if label == target:         
                    pred_prob = prob
            row_pred.append(pred_prob) 
        #print(row_pred)
        #print(max(row_pred))
        pred.append(np.median(row_pred))
        
    return log_loss(val_df[target],pred)
'''

" spliting comment doesn't help\nimport numpy as np\ndef calculate_log_loss_1(model, target):\n    pred = []\n    for index, row in val_df.iterrows():        \n        predict_probas = model.predict_proba(row['sentences'],k=2) \n        #print(row['sentences'])\n        #print(predict_probas)\n        row_pred = []\n        for predict in predict_probas:\n            pred_prob = 0\n            for label, prob in predict:\n                if label == target:         \n                    pred_prob = prob\n            row_pred.append(pred_prob) \n        #print(row_pred)\n        #print(max(row_pred))\n        pred.append(np.median(row_pred))\n        \n    return log_loss(val_df[target],pred)\n"

In [195]:
comment_ids = []
comments = []
for index, row in val_df.iterrows():
    comments.append(row['comment_text'])
    comment_ids.append(row['id'])

In [198]:
predict_probas = {}
param = {}
# 0.134355992393
param['toxic'] = {'maxn': 5, 'lr': 0.1, 'minn': 1, 'epoch': 8, 'wordNgrams':2, 'dim': 100}
param['severe_toxic'] = {'epoch':3, 'maxn':5, 'minn':1, 'wordNgrams':2, 'dim': 100}
param['obscene'] = {'epoch':2, 'maxn':5, 'minn':1, 'wordNgrams':2, 'dim': 100}
param['threat'] = {'epoch':2, 'maxn':4, 'minn':1, 'wordNgrams':2, 'dim': 100}
param['insult'] = {'epoch':2, 'maxn':4, 'minn':1, 'wordNgrams':2, 'dim': 100}
param['identity_hate'] = {'epoch':5, 'maxn':3, 'minn':2, 'wordNgrams':2, 'dim': 100}

In [199]:
target = 'toxic'
model = train_model(param[target], target, all_sample = False)
print(calculate_log_loss(model, target))

0.145851806643


In [118]:
print(calculate_log_loss_1(model, target))

0.169742366782


In [10]:
predict_probas[target] = model.predict_proba(comments,k=2)   

In [11]:
model_df = val_df.copy()
model_df['target'] = target
model_df['pred'] = 0
model_df.drop('label', axis=1, inplace=True)
model_df.drop('clean', axis=1, inplace=True)
pred_col = model_df.columns.get_loc('pred')

for index, comment_id in enumerate(comment_ids):   
    for label_predict, prob in predict_probas[target][index]:
        if label_predict != 'clean':            
            # x = model_df.columns.get_loc('pred')
            model_df.iloc[index, pred_col] = prob    

In [179]:
#model_df[(model_df['toxic'] == 0) & (model_df['comment_text'].str.contains("you")) & (model_df['pred'] > 0.6) & (model_df['pred'] < 0.7)][['id','comment_text','pred']]
#model_df[(model_df['toxic'] == 0) & (model_df['pred'] > 0.9)][['id','comment_text','pred']]
model_df[(model_df['toxic'] == 1) & (model_df['pred'] > 0.9)][['id','comment_text','pred']]
#model_df[(model_df['toxic'] == 0) & (model_df['pred'] > 0.8)][['id','comment_text','pred']]
#model_df[(model_df['toxic'] == 1) & (model_df['pred'] < 0.1)][['id','comment_text','pred']]

,id,comment_text,pred
18886,195706267718,what i mean to say is that im a big fucknig id...,0.998047
90941,949687627873,hiding behind these bitches wikipedians who ke...,0.984375
57978,603990608323,"asshole ! i just read this article , and man ,...",0.998047
12640,130606649386,"fuck your filthy mother in the ass , dry !",1.000000
67534,703346481550,you are a useless prick . die of aids,0.998047
90109,941007383834,"hi , my name is widr and i love to suck some a...",0.900391
36697,382757378387,fuck off mexican bastard ! i will work endeles...,0.998047
1217,12784677751,is it true ? i had heard you like to suck cock...,0.998047
93560,976318236227,scones are cool ard you are gay,0.998047
62085,646476475061,"well , fuckhead seemed a little over the top .",0.998047


In [180]:
model_df[model_df['id']==143209495461]['comment_text'].tolist()

['you are a faggot get a life . . .']

In [187]:
model.predict_proba(["koen franse is gay ! ! !"], k=2)

[[('toxic', 0.9980468749261779), ('clean', 1.9531265169053625e-08)]]